# Capacitated Facility Location Problem - Template

In [2]:
from pulp import *

In [3]:
# Lists (sets / Array) of Customers and Facilities
Customer = [1,2,3,4,5]
Facility = ['Fac-1', 'Fac-2', 'Fac-3']

In [4]:
# Dictionaries of the demands and facilities as well as fixed cost at each facility
Demand = {1 : 80, 2 : 270, 3 : 250, 4 : 160, 5 : 180}
Max_Supply = {'Fac-1' : 500, 'Fac-2' : 500, 'Fac-3' : 500} 
fixed_cost = {'Fac-1' : 1000, 'Fac-2' : 1000, 'Fac-3' : 1000 }

In [5]:
# Dictionary of the transportation cost from each facility to all the customers 
transportation_cost = {'Fac-1' : {1 : 4, 2 : 5, 3 : 6, 4 : 8, 5 : 10},
                       'Fac-2' : {1 : 6, 2 : 4, 3 : 3, 4 : 5, 5 : 8},
                       'Fac-3' : {1 : 9, 2 : 7, 3 : 4, 4 : 3, 5 : 4}
                      }

In [6]:
# Setting the Problem
prob = LpProblem("Capacitated_Facility_Location_Problem", LpMinimize)

In [7]:
# Defining our Desicion Variables
use_facility = LpVariable.dicts("Use Facility", Facility, 0, 1, LpBinary)
ser_customer = LpVariable.dicts("Service", [(i,j) for i in Customer for j in Facility], 0)

In [8]:
use_facility

{'Fac-1': Use_Facility_Fac_1,
 'Fac-2': Use_Facility_Fac_2,
 'Fac-3': Use_Facility_Fac_3}

In [9]:
ser_customer

{(1, 'Fac-1'): Service_(1,_'Fac_1'),
 (1, 'Fac-2'): Service_(1,_'Fac_2'),
 (1, 'Fac-3'): Service_(1,_'Fac_3'),
 (2, 'Fac-1'): Service_(2,_'Fac_1'),
 (2, 'Fac-2'): Service_(2,_'Fac_2'),
 (2, 'Fac-3'): Service_(2,_'Fac_3'),
 (3, 'Fac-1'): Service_(3,_'Fac_1'),
 (3, 'Fac-2'): Service_(3,_'Fac_2'),
 (3, 'Fac-3'): Service_(3,_'Fac_3'),
 (4, 'Fac-1'): Service_(4,_'Fac_1'),
 (4, 'Fac-2'): Service_(4,_'Fac_2'),
 (4, 'Fac-3'): Service_(4,_'Fac_3'),
 (5, 'Fac-1'): Service_(5,_'Fac_1'),
 (5, 'Fac-2'): Service_(5,_'Fac_2'),
 (5, 'Fac-3'): Service_(5,_'Fac_3')}

In [10]:
# Setting the Objective Function
prob += lpSum(fixed_cost[j]*use_facility[j] for j in Facility) + lpSum(transportation_cost[j][i]*ser_customer[(i,j)] for j in Facility for i in Customer)

In [11]:
transportation_cost

{'Fac-1': {1: 4, 2: 5, 3: 6, 4: 8, 5: 10},
 'Fac-2': {1: 6, 2: 4, 3: 3, 4: 5, 5: 8},
 'Fac-3': {1: 9, 2: 7, 3: 4, 4: 3, 5: 4}}

In [12]:
for j in Facility:
    for i in Customer:
       print(transportation_cost[j][i]*ser_customer[(i,j)])

4*Service_(1,_'Fac_1')
5*Service_(2,_'Fac_1')
6*Service_(3,_'Fac_1')
8*Service_(4,_'Fac_1')
10*Service_(5,_'Fac_1')
6*Service_(1,_'Fac_2')
4*Service_(2,_'Fac_2')
3*Service_(3,_'Fac_2')
5*Service_(4,_'Fac_2')
8*Service_(5,_'Fac_2')
9*Service_(1,_'Fac_3')
7*Service_(2,_'Fac_3')
4*Service_(3,_'Fac_3')
3*Service_(4,_'Fac_3')
4*Service_(5,_'Fac_3')


In [13]:
# Costraints
for i in Customer:
    prob += lpSum(ser_customer[(i,j)] for j in Facility) == Demand[i]

for j in Facility:
    prob += lpSum(ser_customer[(i,j)] for i in Customer) <= Max_Supply[j]*use_facility[j]

for i in Customer:
    for j in Facility:
        prob += ser_customer[(i,j)] <= Demand[i]*use_facility[j]

In [71]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/wenyuc/opt/anaconda3/envs/bioze_env/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/w_/w1t73xzx35j4jtrj94fjskj80000gn/T/43316f6ea1c34c5b9c5f71d41abce2e2-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/w_/w1t73xzx35j4jtrj94fjskj80000gn/T/43316f6ea1c34c5b9c5f71d41abce2e2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 28 COLUMNS
At line 116 RHS
At line 140 BOUNDS
At line 144 ENDATA
Problem MODEL has 23 rows, 18 columns and 63 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5610 - 0.00 seconds
Cgl0004I processed model has 23 rows, 18 columns (3 integer (3 of which binary)) and 63 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 2.22045e-16
Cbc0038I Solution found of 5610
Cbc0038I Relaxing continuous gives 5610
Cbc0038I Bef

1

In [72]:
status = LpStatus[prob.status]


In [73]:
print("Solution Status = ", LpStatus[prob.status])

Solution Status =  Optimal


In [74]:
# if status == "Optimal":
#     # Retrieve and store the variable values
#     optimal_x = use_facility.varValue
#     optimal_y = ser_customer.varValue

#     # Retrieve and store the optimal objective value
#     optimal_obj_value = prob.objective.value()
# else:
#     optimal_x = None
#     optimal_y = None
#     optimal_obj_value = None

# print("Optimization Status:", status)
# print("Optimal x:", optimal_x)
# print("Optimal y:", optimal_y)
# print("Optimal Objective Value:", optimal_obj_value)

In [75]:
# Print the status of the optimization
print("Optimization Status:", LpStatus[prob.status])

# Print the optimal values of decision variables
for j in Facility:
    print(f"Use Facility {j}: {use_facility[j].varValue}")

for i in Customer:
    for j in Facility:
        print(f"Service customer {i} from Facility {j}: {ser_customer[(i, j)].varValue}")

Optimization Status: Optimal
Use Facility Fac-1: 0.0
Use Facility Fac-2: 1.0
Use Facility Fac-3: 1.0
Service customer 1 from Facility Fac-1: 0.0
Service customer 1 from Facility Fac-2: 80.0
Service customer 1 from Facility Fac-3: 0.0
Service customer 2 from Facility Fac-1: 0.0
Service customer 2 from Facility Fac-2: 270.0
Service customer 2 from Facility Fac-3: 0.0
Service customer 3 from Facility Fac-1: 0.0
Service customer 3 from Facility Fac-2: 150.0
Service customer 3 from Facility Fac-3: 100.0
Service customer 4 from Facility Fac-1: 0.0
Service customer 4 from Facility Fac-2: 0.0
Service customer 4 from Facility Fac-3: 160.0
Service customer 5 from Facility Fac-1: 0.0
Service customer 5 from Facility Fac-2: 0.0
Service customer 5 from Facility Fac-3: 180.0


In [76]:
# Print the solution of Binary Decision Variables
Tolerance = 0.0001
for j in Facility:
    if use_facility[j].varValue > Tolerance:
        print("Estalish Facility at site = ", j)

Estalish Facility at site =  Fac-2
Estalish Facility at site =  Fac-3


In [77]:
# Print the solution of Continuous Decision Variables
for v in prob.variables():
    print(v.name, "=", v.varValue)

Service_(1,_'Fac_1') = 0.0
Service_(1,_'Fac_2') = 80.0
Service_(1,_'Fac_3') = 0.0
Service_(2,_'Fac_1') = 0.0
Service_(2,_'Fac_2') = 270.0
Service_(2,_'Fac_3') = 0.0
Service_(3,_'Fac_1') = 0.0
Service_(3,_'Fac_2') = 150.0
Service_(3,_'Fac_3') = 100.0
Service_(4,_'Fac_1') = 0.0
Service_(4,_'Fac_2') = 0.0
Service_(4,_'Fac_3') = 160.0
Service_(5,_'Fac_1') = 0.0
Service_(5,_'Fac_2') = 0.0
Service_(5,_'Fac_3') = 180.0
Use_Facility_Fac_1 = 0.0
Use_Facility_Fac_2 = 1.0
Use_Facility_Fac_3 = 1.0


In [78]:
# Initialize lists to store assignment information
assigned_farms = {j: [] for j in Facility}

# Collect assigned farms
for i in Facility:
    for j in Customer:
        if ser_customer[(j,i)].varValue > 0.00001:
            assigned_farms[i].append(j)

assigned_farms

{'Fac-1': [], 'Fac-2': [1, 2, 3], 'Fac-3': [3, 4, 5]}

In [79]:
# Print Optimal
print("Total Cost = ", value(prob.objective))

Total Cost =  5610.0
